<a href="https://colab.research.google.com/github/Icrz11/CS114.K21/blob/master/Sacrasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Giới thiệu về Bài toán nhận biết Sarcasm
  * Trong thực tế có rất nhiều title báo giật tít trên các trang báo lá cải, cũng như mạng xã hội

  * Vì vậy mà việc detect các title này để xác minh tin nào là tin chính thống, tin nào là tin châm biếm đang là vấn đề lớn (phát triển rộng hơn nữa là bài toán về kiểm tra fake news)

  * Dataset được sử dụng trong bài toán này có sẵn ở kaggle, và được thu thập từ 2 nguồn đó là trang web https://www.huffpost.com/ - chuyên tin chính thống, cùng với  https://www.theonion.com/ - Chuyên tin lá cải


# Lấy Data từ Kaggle về

In [5]:
!wget https://raw.githubusercontent.com/Icrz11/CS114.K21/master/Sarcasm_Headlines_Dataset.json

--2020-06-26 12:25:20--  https://raw.githubusercontent.com/Icrz11/CS114.K21/master/Sarcasm_Headlines_Dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5616830 (5.4M) [text/plain]
Saving to: ‘Sarcasm_Headlines_Dataset.json’

Sarcasm_Headlines_D 100%[===================>]   5.36M  14.6MB/s    in 0.4s    

2020-06-26 12:25:20 (14.6 MB/s) - ‘Sarcasm_Headlines_Dataset.json’ saved [5616830/5616830]



In [6]:
#read data
import pandas as pd
df = pd.read_json("/content/Sarcasm_Headlines_Dataset.json", lines = True)
df.head()
df = df.drop(['article_link'], axis = 1)
print (df)

                                                headline  is_sarcastic
0      former versace store clerk sues over secret 'b...             0
1      the 'roseanne' revival catches up to our thorn...             0
2      mom starting to fear son's web series closest ...             1
3      boehner just wants wife to listen, not come up...             1
4      j.k. rowling wishes snape happy birthday in th...             0
...                                                  ...           ...
26704               american politics in moral free-fall             0
26705                            america's best 20 hikes             0
26706                              reparations and obama             0
26707  israeli ban targeting boycott supporters raise...             0
26708                  gourmet gifts for the foodie 2014             0

[26709 rows x 2 columns]


> Dùng pandas đọc file .json 

> Kế đến ta sẽ drop cột article link đi (vì cột này không cần thiết lắm trong bài toán của ta)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26709 entries, 0 to 26708
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   headline      26709 non-null  object
 1   is_sarcastic  26709 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 417.5+ KB


1 số info từ Data ta vừa thu thâp được, trong đó data này có 26709 mẫu bao gồm 2 thông tin là headlibe + nhãn có phải là tin châm biếm hay không

In [26]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
import string 
from string import digits
from string import punctuation
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
for i in df['headline']:
  i = str(i)

Import tất các thư viện cần dùng trong bài toán này

1. Numpy - Xử lí các ma trận
2. Sklearn - Các hàm liên quan đến thuật toán máy học cũng như đánh giá model
3. String - thư viện xử lí các kí tự 

# Tiền xử lí dữ liệu

In [10]:
hl_clean = []
for i in df['headline']:
  clean = i.translate(str.maketrans('', '', punctuation))
  clean = clean.translate(str.maketrans('', '', digits))
  hl_clean.append(clean)

print('Before: ')
print(df['headline'][37])
print ('After: ')
print (hl_clean[37])
for i in hl_clean:
  i = i.lower()


Before: 
'moana' sails straight to the top of the box office with massive $81.1 million opening
After: 
moana sails straight to the top of the box office with massive  million opening


Loại bỏ các thành phần không đóng vai trò quan trọng trong câu của chúng ta

Các thành phần bị loại bỏ:
> Các chữ số 

> Các dấu câu

In [11]:
hl_tokens = []
for i in hl_clean:
  i = i.split()
  hl_tokens.append(i)

print ('before: ')
print (hl_clean[100])
print ('after: ')
print (hl_tokens[100])

before: 
demi lovato drops emotional nightingale music vid
after: 
['demi', 'lovato', 'drops', 'emotional', 'nightingale', 'music', 'vid']


Tách các từ trong câu ra thành các token riêng lẽ ở trên là ví dụ

In [12]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
Lemmantizer = WordNetLemmatizer()
hl_lemma = []
count = 0
print (hl_tokens[10])
for i in (hl_tokens):
  hl_lemma.append([])
  for j in i:
    hl_lemma[count].append(Lemmantizer.lemmatize(j))
  count+=1
print (hl_lemma[10])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
['airline', 'passengers', 'tackle', 'man', 'who', 'rushes', 'cockpit', 'in', 'bomb', 'threat']
['airline', 'passenger', 'tackle', 'man', 'who', 'rush', 'cockpit', 'in', 'bomb', 'threat']


Chuyển toàn bộ từ về form cơ bản của nó

Ví dụ: goes -> go


In [13]:
import nltk
nltk.download ('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words = list(stop_words)
hl_re = []
count = 0
for i in hl_lemma:
  hl_re.append([])
  for j in i:
    if (j not in stop_words):
      hl_re[count].append(j)
  count+=1
print (hl_re)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[['former', 'versace', 'store', 'clerk', 'sue', 'secret', 'black', 'code', 'minority', 'shopper'], ['roseanne', 'revival', 'catch', 'thorny', 'political', 'mood', 'better', 'worse'], ['mom', 'starting', 'fear', 'son', 'web', 'series', 'closest', 'thing', 'grandchild'], ['boehner', 'want', 'wife', 'listen', 'come', 'alternative', 'debtreduction', 'idea'], ['jk', 'rowling', 'wish', 'snape', 'happy', 'birthday', 'magical', 'way'], ['advancing', 'world', 'woman'], ['fascinating', 'case', 'eating', 'labgrown', 'meat'], ['ceo', 'send', 'kid', 'school', 'work', 'company'], ['top', 'snake', 'handler', 'leaf', 'sinking', 'huckabee', 'campaign'], ['friday', 'morning', 'email', 'inside', 'trump', 'presser', 'age'], ['airline', 'passenger', 'tackle', 'man', 'rush', 'cockpit', 'bomb', 'threat'], ['facebook', 'reportedly', 'working', 'healthcare', 'feature', 'apps'], ['north', 'korea', 'pra

In [14]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
Lemmantizer = WordNetLemmatizer()
def Preprocessing_Data (Headline):
  Headline = Headline.lower()
  #Dùng để loại bỏ các dấu câu, chữ số
  clean = Headline.translate(str.maketrans('', '', punctuation))
  clean = clean.translate(str.maketrans('', '', digits))
  #Tách các từ ra thành các token
  clean = clean.split()
  #Chuyển chúng về dạng "root" của chúng
  result = []
  for i in clean:
    result.append(Lemmantizer.lemmatize(i))
  return result

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Đưa dữ liệu vào model để Trainning (sử dụng bag of words để features extraction)

In [15]:
tmp = hl_re
wordfreq = {}
for i in tmp:
  for token in i:
    if token not in wordfreq.keys():
      wordfreq[token] = 1
    else:
      wordfreq[token] += 1


In [16]:
sort_wordfreq = sorted(wordfreq.items(), key=lambda x: x[1], reverse=True)


In [17]:
vector_len = 2000
tmp2 = []
for i in range (vector_len):
  tmp2.append(sort_wordfreq[i][0])

In [18]:
sentence_vectors = []
for sentence in hl_re:
  sent_vec = np.zeros(shape = (vector_len,))
  for i,token in enumerate(tmp2):
    if token in sentence:
      sent_vec[i] = 1
  sentence_vectors.append(sent_vec)

In [ ]:
def Bag_Of_Word(Headline):
  sent_vec = np.zeros(shape = (vector_len,))
  for i in range (len(Headline)):
    if (Headline[i] in tmp2):
      sent_vec[i] = 1
    return sent_vec

In [32]:
Y = df.iloc[:,1]
X = sentence_vectors
X = np.array (X)
Y = np.array(Y)
print (len(X))
print (len(Y))

26709
26709


In [33]:
X_train, X_test, y_train, y_test = train_test_split (X,Y, test_size = 0.2, random_state = 30)


### Naive Bayes

In [24]:
from sklearn.naive_bayes import GaussianNB
model1 = GaussianNB()
model1.fit (X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [25]:
re1 = model1.predict (X_test)

In [27]:
confusion_matrix(y_test, re1)

array([[1817, 1190],
       [ 426, 1909]])

Từ confusion matrix ta thấy được số nhãn được:

1817 -> Số nhãn được dự đoán đúng thuộc loại negative | 1190 -> Số nhãn được dự đoán sai loại positive

426 -> Số nhãn được dự đoán sai loại Positive | 1909 -> Số nhãn được dự đoán đúng như Positive

Với negative = 0 và positive = 1 (0 là non sacrasm và 1 là sacrasm)

In [31]:
f1_model1 = f1_score(y_test, re1)
print (f1_model1)

0.7026131762973868


Ta thấy chỉ số F1-score ở model này khá cao (0.70) với F1 score được tính bởi 

$ (2*Precision*Recall) 
/ (Precision + Recall) $

Cho nên nếu cả precision và recall đều cao thì F1 - score mới cao được


      > $Precision = 1909 / (1909 + 426) = 0.817$

      > $Recall = 1909 / (1909 + 1190) = 0.61$

Precision biểu thị cho việc tiên đoán đúng nhiều nhãn

### K-NN


In [34]:
from sklearn.neighbors import KNeighborsClassifier
model2 = KNeighborsClassifier()
model2.fit (X_train, y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [36]:
re2 = model2.predict (X_test)

In [39]:
confusion_matrix(y_test, re2)

array([[2032,  975],
       [ 998, 1337]])

In [41]:
f1_model2 = f1_score(y_test, re2)
print (f1_model2)

0.5754250053798149


### SVM

In [ ]:
from sklearn import svm
model3 = svm.SVC()
model3.fit (X_train, y_train)

In [ ]:
re3 = model3.predict (X_test)

# Đưa dữ liệu gồm 2k headline được crawl về để test model



In [21]:
!wget https://raw.githubusercontent.com/Icrz11/CS114.K21/master/test_data.csv

--2020-06-26 12:30:11--  https://raw.githubusercontent.com/Icrz11/CS114.K21/master/test_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 165600 (162K) [text/plain]
Saving to: ‘test_data.csv’

test_data.csv       100%[===================>] 161.72K  --.-KB/s    in 0.04s   

2020-06-26 12:30:12 (3.73 MB/s) - ‘test_data.csv’ saved [165600/165600]



In [22]:
test = pd.read_csv ('/content/test_data.csv')
test.head()

,headline,is_sarcastic
0,BirthdayForBreonna Marks What Would've Been Br...,0
1,TheShowMustBePaused Was Eclipsed By #BlackOutT...,0
2,"Again, Again, Again!' Exclaims Clapping, Grinn...",1
3,"And Then There Were 23,' Says Wayne Messam Cro...",1
4,And Then Those 12 People Send It To 12 People ...,1


In [ ]:
Y = test['is_sarcastic']
Y = np.array(Y)
X = test['headline']

In [ ]:
X.head()

0    BirthdayForBreonna Marks What Would've Been Br...
1    TheShowMustBePaused Was Eclipsed By #BlackOutT...
2    Again, Again, Again!' Exclaims Clapping, Grinn...
3    And Then There Were 23,' Says Wayne Messam Cro...
4    And Then Those 12 People Send It To 12 People ...
Name: headline, dtype: object

In [ ]:
X_train = []
for i in X:
  X_train.append(Preprocessing_Data(i))
kHeadline = []
for i in X_train:
  kHeadline.append(Bag_Of_Word(i))

In [ ]:
model = joblib.load('/content/SVMclassifiSacrasm.pkl')
model_pred = model.predict (kHeadline)

In [ ]:
acc = accuracy_score (Y, model_pred)
print (acc)

0.5354330708661418


Sử dụng Bag of word

Ở đây có thể thấy ACC tiên đoán trên tập dữ liệu vừa crawl về và rút trich đặc trưng bằng BOW khá thấp, có thể vì trong các headline mới này chứa các từ mới chưa có trong từ điển của BOW dẫn đến hiệu suất rút trích đặc trưng thấp cũng như hiệu quả khi đưua vào máy học thấp

# Chương trình đưa vào 1 Headline và dự đoán

In [ ]:
#Bag of word
Inp = input ('Moi nhap headline: ')
Inp = Preprocessing_Data(Inp)
print (Inp)
Inp = Bag_Of_Word(Inp)
Inp = np.array(Inp)
Kqua = model.predict(Inp)
print (Kqua)

Moi nhap headline: Trump Tackles Medical Supply Shortage By Awarding ExxonMobil Contract To Drill For Ventilators In Arctic
['trump', 'tackle', 'medical', 'supply', 'shortage', 'by', 'awarding', 'exxonmobil', 'contract', 'to', 'drill', 'for', 'ventilator', 'in', 'arctic']


# Trả lời câu hỏi


1. Mô tả bài toán ở trên cùng

2. Thu thâp headline ở 2 trang web trên, em sử dụng 1 tool extentions ở gooogle chrome có tên là Data miner để crawl dữ liệu

3. Chúng em sử dụng 2 phương pháp để rút trích đặc trưng là Bag of Words (túi từ - sử dụng khoảng 2000 từ thường xuất hiện nhất) và TFIDF

4. Em sử test trên 4 thuật toán là KNN, Naive Bayes, svm, Decision tree và thấy model khi dùng svm là cho tỉ lệ dự đoán cao nhất khoảng 77% trên tập test đc chia ra từ Data, em định sử dụng grid search cho svm nhưng thấy hơi ngu (vì svm train khá lâu) nên em vẫn chưa tinh chỉnh tham số cho model

5. Model đã train sẽ được lưu lại thông qua thư viện joblib, chương trình đưa 1 headline mới vào để dự đoán được viết ở phía trên nhưng chỉ dùng bag of word để rút trích đặt trưng chứ em chưa dùng thử tf-idf


6. Performance trên test set (được tách ra từ Data) là khoảng 77% nhưng áp dụng model trên Data mới đc crawl về (2k headline) lại khá thấp (53%), có thể nguyên do là vì trong data mới có các từ không có trong từ điển của bag of word, hoặc có thể do model của em bị over fitting.
